In [1]:
#!pip install kats

In [1]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

---

In [2]:
from kats.tsfeatures.tsfeatures import TsFeatures
from kats.consts import TimeSeriesData, TimeSeriesIterator
from utils import get_data
df_emg = get_data()

memory usage:  96.13 MB


In [3]:
import pandas as pd
import numpy as np

## testing time type

In [4]:
ts_data = TimeSeriesData(value=df_emg, time=pd.Series(np.arange(0, len(df_emg))))

In [21]:
ts_data.time

0         1970-01-01 00:00:00.000000000
1         1970-01-01 00:00:00.000000001
2         1970-01-01 00:00:00.000000002
3         1970-01-01 00:00:00.000000003
4         1970-01-01 00:00:00.000000004
                       ...             
3599995   1970-01-01 00:00:00.003599995
3599996   1970-01-01 00:00:00.003599996
3599997   1970-01-01 00:00:00.003599997
3599998   1970-01-01 00:00:00.003599998
3599999   1970-01-01 00:00:00.003599999
Length: 3600000, dtype: datetime64[ns]

## default example

In [4]:
ts_data_list = [TimeSeriesData(value=df_emg[c], time=df_emg.index) for c in df_emg.columns]

In [5]:
fs = 1000
fe = TsFeatures(
    window_size=fs * 30,
    window=30 * fs,
    selected_features=['mean', 'var', 'length']
)

out = [fe.transform(ts) for ts in ts_data_list]

In [9]:
ts_data_list[0]

,time,emg
0,2021-06-30 07:05:52.620232,0.00000
1,2021-06-30 07:05:52.621232,0.00000
2,2021-06-30 07:05:52.622232,0.00000
3,2021-06-30 07:05:52.623232,0.00000
4,2021-06-30 07:05:52.624232,0.00000
...,...,...
3599995,2021-06-30 08:05:52.615232,0.34375
3599996,2021-06-30 08:05:52.616232,0.34375
3599997,2021-06-30 08:05:52.617232,0.34375
3599998,2021-06-30 08:05:52.618232,0.34375


In [8]:
out

[{'length': 3600000, 'mean': 0.6727282404899597, 'var': 0.07464560121297836},
 {'length': 3600000, 'mean': -0.09149625897407532, 'var': 0.20476381480693817},
 {'length': 3600000, 'mean': 0.2712729275226593, 'var': 0.17504480481147766},
 {'length': 3600000, 'mean': 0.6727282404899597, 'var': 0.07464560121297836},
 {'length': 3600000, 'mean': -0.09149625897407532, 'var': 0.20476381480693817}]

In [7]:
pd.DataFrame(out).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   length  5 non-null      int64  
 1   mean    5 non-null      float64
 2   var     5 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 248.0 bytes


In [8]:
# fs = 1000
TsFeatures(
#     window_size=fs * 30,
#     window=30 * fs,
    selected_features=['mean', 'var', 'length']
).transform(ts_data)

TypeError: unhashable type: 'dict'

In [27]:
fe.

TypeError: unhashable type: 'dict'

In [4]:
import sys
sys.path.append('../../')
from tsflex.features import FeatureCollection, NumpyFuncWrapper
from tsflex.features import FeatureDescriptor, MultipleFeatureDescriptors

#----------------------------------------------------------------------------
quantiles = [0.25, 0.5, 0.75]


def type_wrapper(x: np.ndarray, type_wrapped_func, **kwargs):
    return type_wrapped_func(x, **kwargs).astype(x.dtype)


# -- 2. in-line functions
#    You can define your functions locally; these will serialize flawleslly
def slope(x):
    return np.polyfit(np.arange(0, len(x)), x, 1)[0]


f_slope = NumpyFuncWrapper(type_wrapper, output_names="slope", type_wrapped_func=slope)

# -- 3. Lambda's
#    Or even use lambda's and other modules' functions
def rms(x): return np.sqrt(np.mean(x**2))
f_rms = NumpyFuncWrapper(rms, output_names="rms")
# f_rms = NumpyFuncWrapper(lambda x: np.sqrt(np.mean(x ** 2)), output_names="rms")
f_area = NumpyFuncWrapper(np.sum, output_names="area")

# (For convenience) we store the constructed `NumpyFuncWrappers` in a list
segment_funcs = [
    np.min,
    np.max,
    np.mean,
    np.std,
    np.var,
    ss.skew,
    ss.kurtosis,
    f_slope,
    f_rms,
    f_area,
]

In [5]:
from utils import get_data
df_emg = get_data()

memory usage:  96.13 MB


In [8]:
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects

In [8]:
# fc = FeatureCollection(
#     feature_descriptors=[
#         MultipleFeatureDescriptors(
#             functions=segment_funcs,
#             series_names=["emg", "eog", "lso", "rio", "m1-a1"],
#             windows=["30s"],
#             strides=["10s"],
#         )
#     ]
# )
# out = fc.calculate(data=df_emg, n_jobs=None, return_df=True, show_progress=True)

In [6]:
fc = FeatureCollection(
    feature_descriptors=[
        MultipleFeatureDescriptors(
            functions=segment_funcs,
            series_names=["emg", "eog", "lso", "rio", "m1-a1"],
            windows=["30s"],
            strides=["10s"],
        )
    ]
)

In [7]:
%mprun -f fc.calculate fc.calculate(data=df_emg, n_jobs=10, return_df=True, show_progress=True)

Filename: ../../tsflex/features/feature_collection.py

Line #    Mem usage    Increment  Occurences   Line Contents
   170    327.0 MiB    327.0 MiB           1       def calculate(
   171                                                 self,
   172                                                 data: Union[pd.Series, pd.DataFrame, List[Union[pd.Series, pd.DataFrame]]],
   173                                                 return_df: Optional[bool] = False,
   174                                                 window_idx: Optional[str] = 'end',
   175                                                 approve_sparsity: Optional[bool] = False,
   176                                                 show_progress: Optional[bool] = False,
   177                                                 logging_file_path: Optional[Union[str, Path]] = None,
   178                                                 n_jobs: Optional[int] = None,
   179                                             ) -> Union